# Combine national and NERC data
Take the various file (EIA facilities, EIA state-level totals, EPA emissions, etc) and combine them to determine total generation, CO₂ emissions, etc nationally and at NERC regions.

In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json

idx = pd.IndexSlice
cwd = os.getcwd()
data_path = join(cwd, '..', 'Data storage')

In [2]:
%load_ext watermark
%watermark -iv -v

json        2.0.9
yapf        0.16.3
pandas      0.20.2
numpy       1.13.1
CPython 3.6.2
IPython 6.2.1


In [3]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [4]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [5]:
%aimport Data.make_data
from Data.make_data import states_in_nerc
%aimport Analysis.index
from Analysis.index import facility_emission_gen, group_facility_data
%aimport Analysis.index
from Analysis.index import facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport util.utils
from util.utils import rename_cols, add_facility_location

## Load data

In [6]:
cwd = os.getcwd()
path = join(data_path, 'Facility gen fuels and CO2 2017-08-31.zip')
eia_fac = pd.read_csv(path)

In [7]:
rename_cols(eia_fac)

In [8]:
eia_fac.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,M,SUB,6,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-06-01,2,0.0,0.0,0.0,0.0
1,M,SUB,5,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-05-01,2,0.0,0.0,0.0,0.0
2,M,SUB,4,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-04-01,2,0.0,0.0,0.0,0.0
3,M,SUB,3,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-03-01,1,0.0,0.0,0.0,0.0
4,M,SUB,2,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-02-01,1,0.0,0.0,0.0,0.0


In [9]:
path = join(data_path, 'Derived data',
            'Monthly EPA emissions 2017-08-31.csv')
epa = pd.read_csv(path)

## Adjusted facility emissions and generation
I have both the state and custom fuel categories here, but am only using the state fuel categories for now.

In [10]:
fuel_cat_folder = join(data_path, 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

In [113]:
co2, gen_fuels_custom = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=False)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories
Gen/fuels to custom categories


In [11]:
co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=True)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [12]:
co2.head()

,year,month,plant id,final co2 (kg)
0,2001,1,2,0.000000e+00
1,2001,1,3,9.625411e+08
2,2001,1,4,0.000000e+00
3,2001,1,7,6.700092e+07
4,2001,1,8,7.530204e+08


In [13]:
gen_fuels_state.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,COW,2001,1,3,8275496.0,852306.0,8275496.0,31.006900,-88.010300,1,772103776.8,772103776.8,772103776.8,772103776.8
1,COW,2001,1,7,587162.0,45991.0,587162.0,34.012800,-85.970800,1,54782214.6,54782214.6,54782214.6,54782214.6
2,COW,2001,1,8,6910786.0,706857.0,6910786.0,33.644344,-87.196486,1,644776333.8,644776333.8,644776333.8,644776333.8
3,COW,2001,1,10,3108892.0,320454.0,3108892.0,32.601700,-87.781100,1,290059623.6,290059623.6,290059623.6,290059623.6
4,COW,2001,1,26,9745127.0,1005460.0,9745127.0,33.244211,-86.458056,1,909220349.1,909220349.1,909220349.1,909220349.1


### Check generation and fuel consumption totals

Interesting - there is some small part of generation that I'm losing along the way. It's 5 orders of magnitude smaller than the total though.

In [15]:
eia_fac['generation (mwh)'].sum()

65594379743.764412

In [16]:
gen_fuels_state['generation (mwh)'].sum()

65594327524.764351

Fuel consumption is pretty identical though

In [17]:
eia_fac['total fuel (mmbtu)'].sum(), eia_fac['elec fuel (mmbtu)'].sum()

(686393145435.78979, 645653921528.08057)

In [18]:
gen_fuels_state['total fuel (mmbtu)'].sum(), gen_fuels_state['elec fuel (mmbtu)'].sum()

(686393145435.78979, 645653921528.0802)

## Extra gen/fuels from non-reporting

In [14]:
%aimport Analysis.index
from Analysis.index import extra_emissions_gen

Total EIA generation/fuel consumption and emission factors

In [15]:
cwd = os.getcwd()
path = join(data_path,
            'EIA country-wide gen fuel CO2 2017-08-31.csv')
eia_total = pd.read_csv(path)

path = join(data_path,
            'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [16]:
eia_total.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
0,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,6,2524250.11,2017-06-01,2,NaN,NaN,0.0,0.0
1,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,5,2479480.14,2017-05-01,2,NaN,NaN,0.0,0.0
2,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,4,2267811.28,2017-04-01,2,NaN,NaN,0.0,0.0
3,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,3,2054274.30,2017-03-01,1,NaN,NaN,0.0,0.0
4,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,2,1449156.85,2017-02-01,1,NaN,NaN,0.0,0.0


### Calculate CO₂, generation, and fuel consumption that is not captured by facilities

In [17]:
extra_co2, extra_gen_fuel = extra_emissions_gen(gen_fuels_state, eia_total, ef)

Results match what I have previously found in the notebooks up on GitHub (Emissions Index repo)

In [18]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16918390.0       1011665.061         13157576.0
          2              14968346.0        911346.865         11574491.0
          3              18321543.0       1153328.913         14765162.0
          4              13530117.0        776341.623         10374008.0
          5              14014024.0        801266.923         10920463.0

In [19]:
extra_gen_fuel.loc[idx['WND',:,:]]

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
WND  2001 1           -3.446894e+06      5.566881e+04      -3.446894e+06
          2           -3.288177e+06      1.130211e+05      -3.288177e+06
          3           -4.798975e+06      6.768812e+04      -4.798975e+06
          4           -6.166244e+06      8.794247e+04      -6.166244e+06
          5           -5.694490e+06      8.393153e+04      -5.694490e+06
          6           -6.120034e+06      7.735354e+04      -6.120034e+06
          7           -5.827621e+06      7.101049e+04      -5.827621e+06
          8           -5.409429e+06      5.369600e+04      -5.409429e+06
          9           -4.550627e+06      4.983153e+04      -4.550627e+06
          10          -5.668965e+06      5.820182e+04      -5.668965e+06
          11          -4.551249e+06      2.999030e+04      -4.551249e+06
          12          -6.077985e+06      2.744590e+04      -6.077985e+06
     2002 1           -8.237968e+06      1.614509e+03      -8.237968e+06
          2           -7.247621e+06      1.495079e+03      -7.247621e+06
          3           -8.650071e+06      1.805048e+03      -8.650071e+06
          4           -1.040601e+07      1.512223e+03      -1.040601e+07
          5           -1.094733e+07      1.524876e+03      -1.094733e+07
          6           -1.143913e+07      1.855552e+03      -1.143913e+07
          7           -9.036664e+06      2.156177e+03      -9.036664e+06
          8           -9.914911e+06      2.061015e+03      -9.914911e+06
          9           -7.466379e+06      1.887244e+03      -7.466379e+06
          10          -7.453861e+06      1.644759e+03      -7.453861e+06
          11          -6.657461e+06      1.521470e+03      -6.657461e+06
          12          -7.665064e+06      1.722605e+03      -7.665064e+06
     2003 1           -6.475789e+06      4.380000e-01      -6.475789e+06
          2           -7.633388e+06     -2.490000e-01      -7.633388e+06
          3           -1.061074e+07     -2.510000e-01      -1.061074e+07
          4           -1.119102e+07     -2.110000e-01      -1.119102e+07
          5           -1.030637e+07     -2.610000e-01      -1.030637e+07
          6           -1.072381e+07     -3.660000e-01      -1.072381e+07
...                             ...               ...                ...
     2015 1           -1.412664e+08      3.183256e+03      -1.412664e+08
          2           -1.390198e+08      3.654121e+03      -1.390198e+08
          3           -1.426223e+08      3.460416e+03      -1.426223e+08
          4           -1.664585e+08      4.872189e+03      -1.664585e+08
          5           -1.597837e+08      5.320185e+03      -1.597837e+08
          6           -1.250309e+08      4.493551e+03      -1.250309e+08
          7           -1.273987e+08      4.591517e+03      -1.273987e+08
          8           -1.218547e+08      4.083333e+03      -1.218547e+08
          9           -1.299189e+08      3.027033e+04      -1.299189e+08
          10          -1.526123e+08      3.567217e+03      -1.526123e+08
          11          -1.833724e+08      4.449315e+03      -1.833724e+08
          12          -1.872415e+08      5.930371e+03      -1.872415e+08
     2016 1           -1.666799e+08      6.453995e+05      -1.666799e+08
          2           -1.818505e+08      6.895701e+05      -1.818505e+08
          3           -1.977699e+08      7.570480e+05      -1.977699e+08
          4           -1.865983e+08      7.213771e+05      -1.865983e+08
          5           -1.691026e+08      6.494653e+05      -1.691026e+08
          6           -1.467114e+08      5.751191e+05      -1.467114e+08
          7           -1.585294e+08      5.831914e+05      -1.585294e+08
          8           -1.221785e+08      4.499372e+05      -1.221785e+08
          9           -1.480642e+08      5.419517e+05      -1.480642e+08
          10          -1.837908e+08      6.582283e+05      -1.837908e+08
  

## Total CO₂ (national)
Combine adjusted CO₂ at facilities and CO₂ from fuel consumption that is not captured by facility data.

In [20]:
facility_co2 = co2.groupby(['year', 'month']).sum()

In [21]:
facility_co2.tail()

plant id  final co2 (kg)
year month                          
2017 2      57482310    1.165442e+11
     3      58485598    1.237187e+11
     4      58955690    1.133454e+11
     5      59223601    1.292581e+11
     6      59451819    1.512113e+11

In [22]:
extra_co2.loc[idx['NG', :, :],:].tail(n=11)

all fuel co2 (kg)  elec fuel co2 (kg)
type year month                                       
NG   2016 8           6.022574e+09        4.039321e+09
          9           4.917984e+09        3.118869e+09
          10          4.312456e+09        2.624005e+09
          11          4.231444e+09        2.458983e+09
          12          4.418713e+09        2.530048e+09
     2017 1           7.823823e+09        6.101011e+09
          2           6.601814e+09        5.030973e+09
          3           8.630216e+09        6.986014e+09
          4           7.823192e+09        6.280070e+09
          5           8.749634e+09        7.244919e+09
          6           1.000377e+10        8.465898e+09

In [23]:
national_co2 = (facility_co2.loc[:, 'final co2 (kg)']
                + extra_co2.loc[:, 'elec fuel co2 (kg)'].groupby(['year', 'month']).sum())

## National Index and gen by fuels

In [24]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16918390.0       1011665.061         13157576.0
          2              14968346.0        911346.865         11574491.0
          3              18321543.0       1153328.913         14765162.0
          4              13530117.0        776341.623         10374008.0
          5              14014024.0        801266.923         10920463.0

In [25]:
gen_fuels_state.groupby(['year', 'month']).sum().tail()

plant id  total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)  \
year month                                                                      
2017 2      89637881        2.471045e+09      2.462247e+08       2.349592e+09   
     3      90951249        2.659571e+09      2.654742e+08       2.526846e+09   
     4      91388289        2.424457e+09      2.422983e+08       2.304171e+09   
     5      91583293        2.676784e+09      2.681190e+08       2.555053e+09   
     6      91677954        2.997427e+09      3.021853e+08       2.876122e+09   

                      lat            lon  quarter  all fuel fossil co2 (kg)  \
year month                                                                    
2017 2      120729.133612 -296504.534805     3192              1.209563e+11   
     3      121757.578709 -298889.971201     3217              1.285636e+11   
     4      121805.826865 -299282.190963     6440              1.176367e+11   
     5      121961.825569 -299520.312134     6446              1.334064e+11   
     6      121896.867105 -299481.456675     6444              1.544630e+11   

            elec fuel fossil co2 (kg)  all fuel total co2 (kg)  \
year month                                                       
2017 2                   1.155701e+11             1.273938e+11   
     3                   1.225800e+11             1.354570e+11   
     4                   1.122119e+11             1.239456e+11   
     5                   1.278850e+11             1.397844e+11   
     6                   1.490479e+11             1.609368e+11   

            elec fuel total co2 (kg)  
year month                            
2017 2                  1.180525e+11  
     3                  1.252646e+11  
     4                  1.147064e+11  
     5                  1.304835e+11  
     6                  1.516790e+11

### Total national generation (all fuels)
Add generation/fuel consumption reported by facilities (with state-level fuel codes) and extra generation/fuel consumption (which also uses state-level fuel codes).

In [26]:
national_gen = (gen_fuels_state
                .groupby(['type', 'year', 'month'])['generation (mwh)'].sum()
                .add(extra_gen_fuel['generation (mwh)'], fill_value=0))

In [27]:
national_gen.groupby(['year', 'month']).sum().tail()

year  month
2017  2        2.898631e+08
      3        3.199879e+08
      4        2.959468e+08
      5        3.236810e+08
      6        3.582983e+08
Name: generation (mwh), dtype: float64

#### Regroup generation from state codes to my custom fuel codes

In [28]:
national_gen = group_fuel_cats(national_gen.reset_index(), custom_fuel_cat,
                               'type', 'fuel category').set_index(['fuel category', 'year', 'month'])

In [29]:
national_gen.tail()

generation (mwh)
fuel category year month                  
Wind          2017 2           21691634.88
                   3           25598907.50
                   4           25403089.66
                   5           22326395.36
                   6           19428938.56

In [30]:
total_gen = national_gen.groupby(['year', 'month']).sum()
total_gen.head()

generation (mwh)
year month                  
2001 1           332493160.0
     2           282940198.0
     3           300706544.0
     4           278078871.0
     5           300491621.0

### Percent generation by custom fuel type

In [36]:
df_list = []
for fuel in national_gen.index.get_level_values('fuel category').unique():
    percent_gen = national_gen.loc[fuel].divide(total_gen, fill_value=0)
    percent_gen['fuel category'] = fuel
    percent_gen.set_index('fuel category', inplace=True, append=True)
    df_list.append(percent_gen)
percent_gen = pd.concat(df_list)

In [162]:
path = join(data_path, 'National data', 'National generation.csv')
national_gen.to_csv(path)

path = join(data_path, 'National data', 'National percent gen.csv')
percent_gen.to_csv(path)

## Facility state and lat/lon file generation
~~Create a .csv with lat/lon and state code for each facility. Need to manually add the NERC region label.~~ ~~I've done this with a spatial join between the lat/lon and NERC shapefiles in QGIS. It should be possible to do the spatial join in [GeoPandas](http://geopandas.org/).  Could also add custom region labels (e.g. eGRID subregions, ISO/RTO boundaries, etc).~~

This isn't possible with a spatial join because NERC regions aren't strictly based on geography. Instead, I've taken the NERC codes from EIA-860 and assigned unknown plants (mostly those that retired before 2012 when modern NERCs were mostly defined, and those that were assigned a plant id in 2017) NERC labels using a k-nearest neighbors algorithm and lat/lon information.

See the `Assign NERC region labels` notebook for the creation of this file.

## Fraction of estimated gen/fuels in each NERC region

**NOTE** EIA changed the reporting requirements for wind and solar facilities in 2017. I'm now going to use the list of annual facilities from 2017 rather than from 2015, but still use the reported generation by those facilities in 2015.

In [32]:
%aimport Data.make_data
from Data.make_data import get_annual_plants

In [33]:
annual_ids_2015 = get_annual_plants(2015)

In [34]:
annual_ids_2017 = get_annual_plants(2017)

Combine the lists of annual plants in 2015 and 2017. This lets us catch facilities that have gone from monthly to annual since 2015, but it also includes plants that were annual in 2015 an may have retired.

There is the possibility of an error in allocation to NERC regions for 2016 state-level generation when using the facilities that changed to annual in 2017. But since the state-level generation is so much smaller in 2016 I don't think this is much of an issue.

In [35]:
annual_ids = set(annual_ids_2015.tolist() + annual_ids_2017.tolist())

In [36]:
len(annual_ids)

7146

### Add NERC region labels

In [37]:
%aimport Analysis.state2nerc
from Analysis.state2nerc import fraction_state2nerc, add_region

In [38]:
cwd = os.getcwd()
path = join(data_path, 'Facility labels',
            'Facility locations_knn.csv')
location_labels = pd.read_csv(path)

In [39]:
nerc_state_path = join(data_path, 'Derived data',
                  'NERC_states.json')

with open(nerc_state_path, 'r') as f:
    nerc_states = json.load(f)

Added the filter that year must be 2015 - was getting all 2015 annual plants, but for all years!

In [40]:
eia_2015_annual = eia_fac.loc[(eia_fac['plant id'].isin(annual_ids)) & 
                              (eia_fac['year'] == 2015)].copy()

# Group to state-level fuel categories
eia_2015_annual = group_fuel_cats(eia_2015_annual, state_fuel_cat)

In [41]:
eia_2015_annual_nerc = add_facility_location(eia_2015_annual, location_labels, 
                                        labels=['state', 'nerc'])

This is 2015 data on annual reporting facilities (from both 2015 and 2017)

In [42]:
eia_2015_annual_nerc.tail()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),state,nerc
92450,WWW,2015,8,58707,211987.0,13966.0,211987.0,33.238611,-80.450278,3,0.0,0.0,19884380.6,19884380.6,SC,RFC
92451,WWW,2015,9,58707,214505.0,13769.0,214505.0,33.238611,-80.450278,3,0.0,0.0,20120569.0,20120569.0,SC,RFC
92452,WWW,2015,10,58707,162426.0,8554.0,162426.0,33.238611,-80.450278,4,0.0,0.0,15235558.8,15235558.8,SC,RFC
92453,WWW,2015,11,58707,245845.0,13352.0,245845.0,33.238611,-80.450278,4,0.0,0.0,23060261.0,23060261.0,SC,RFC
92454,WWW,2015,12,58707,249553.0,14099.0,249553.0,33.238611,-80.450278,4,0.0,0.0,23408071.4,23408071.4,SC,RFC


In [43]:
# Get a list of all state abbreviations

all_states = []
for value in nerc_states.values():
    all_states.extend(value)
all_states = set(all_states)

In [44]:
df_list = []

for state in all_states:
    try:
        df_list.append(fraction_state2nerc(eia_2015_annual_nerc,
                       state, region_col='nerc', fuel_col='type'))
    except:
        print(state)
        pass

In [45]:
nerc_fraction = pd.concat(df_list)
nerc_fraction.set_index(['state', 'nerc', 'type'], inplace=True)
nerc_fraction.sort_index(inplace=True)

In [46]:
nerc_fraction.loc[idx['TX', :, 'WND'], :]

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    SPP  WND       0.125231      0.125231     0.125231
      TRE  WND       0.874769      0.874769     0.874769
      WECC WND       0.000000      0.000000     0.000000

With the values below I can allocate extra state-level generation and fuel use to each of the NERC regions!

In [47]:
nerc_fraction.loc['TX']

% generation  % total fuel  % elec fuel
nerc type                                         
SERC HYC       0.260935      0.260935     0.260935
     NG        0.124994      0.153379     0.092145
     OOG       0.151901      0.546518     0.082765
     OTH      -0.006302      0.025416     0.006614
     WWW      -0.492566      0.190640     0.257302
SPP  NG        0.160722      0.174954     0.185692
     OOG       0.282734      0.159436     0.284516
     PEL       0.478368      0.745665     0.290000
     WAS       0.892327      0.958075     0.847920
     WND       0.125231      0.125231     0.125231
     WWW       0.609573      0.282686     0.398931
TRE  HYC       0.739065      0.739065     0.739065
     NG        0.652469      0.615575     0.650881
     OOG       0.565365      0.294046     0.632719
     OTH       1.006302      0.974584     0.993386
     PC        1.000000      1.000000     1.000000
     PEL       0.487887      0.245383     0.677393
     SUN       0.932089      0.932089     0.932089
     WAS       0.107673      0.041925     0.152080
     WND       0.874769      0.874769     0.874769
     WWW       0.882992      0.526674     0.343767
WECC NG        0.061815      0.056091     0.071281
     PEL       0.033746      0.008952     0.032607
     SUN       0.067911      0.067911     0.067911
     WND       0.000000      0.000000     0.000000

In [48]:
nerc_fraction.tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
WY    WECC NG             1.0           1.0          1.0
           OOG            1.0           1.0          1.0
           OTH            1.0           1.0          1.0
           PEL            1.0           1.0          1.0
           WND            1.0           1.0          1.0

Making sure that no values are greater than 1 (within tolerance)

In [61]:
(nerc_fraction.groupby(['state', 'type']).sum() > 1.0001).any()

% generation    False
% total fuel    False
% elec fuel     False
dtype: bool

In [62]:
(nerc_fraction.groupby(['state', 'type']).sum()
 .loc[(nerc_fraction.groupby(['state', 'type']).sum() > 1).any(axis=1)])

,,% generation,% total fuel,% elec fuel
state,type,,,
AR,NG,1.0,1.0,1.0
TX,PEL,1.0,1.0,1.0


## Allocate extra gen from the state-level to regions

I still need to generate state-level total generation and fuel use!

In [63]:
idx = pd.IndexSlice

In [64]:
# a dictionary to match column names
nerc_frac_match = {'% generation': 'generation (mwh)',
                   '% total fuel': 'total fuel (mmbtu)',
                   '% elec fuel': 'elec fuel (mmbtu)'}

### Load state-level total gen/fuel consumption

In [65]:
path = join(data_path, 'EIA state-level gen fuel CO2.csv')

state_total = pd.read_csv(path, parse_dates=['datetime'])

In [66]:
rename_cols(state_total)
state_total['state'] = state_total['geography'].str[-2:]

In [67]:
state_total.head()

,type,year,month,geography,end,f,last_updated,sector,series_id,start,units,generation (mwh),total fuel (mmbtu),elec fuel (mmbtu),all fuel co2 (kg),elec fuel co2 (kg),datetime,quarter,state
0,AOR,2001,1,USA-AK,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AK-99.M,200101.0,megawatthours,87.00,NaN,NaN,NaN,NaN,2001-01-01,1,AK
1,AOR,2001,1,USA-AL,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AL-99.M,200101.0,megawatthours,401167.59,NaN,NaN,NaN,NaN,2001-01-01,1,AL
2,AOR,2001,1,USA-AR,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AR-99.M,200101.0,megawatthours,136530.37,NaN,NaN,NaN,NaN,2001-01-01,1,AR
3,AOR,2001,1,USA-AZ,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AZ-99.M,200101.0,megawatthours,453.00,NaN,NaN,NaN,NaN,2001-01-01,1,AZ
4,AOR,2001,1,USA-CA,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-CA-99.M,200101.0,megawatthours,1717398.41,NaN,NaN,NaN,NaN,2001-01-01,1,CA


In [70]:
state_total.dtypes

type                          object
year                           int64
month                          int64
geography                     object
end                          float64
f                             object
last_updated                  object
sector                       float64
series_id                     object
start                        float64
units                         object
generation (mwh)             float64
total fuel (mmbtu)           float64
elec fuel (mmbtu)            float64
all fuel co2 (kg)            float64
elec fuel co2 (kg)           float64
datetime              datetime64[ns]
quarter                        int64
state                         object
dtype: object

Simplify the dataframe

In [68]:
cols = list(nerc_frac_match.values())
state_total = state_total.groupby(['state', 'year', 'month', 'type'])[cols].sum()
state_total.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     AOR               87.0                 NaN                NaN
                 COW            46903.0           1120000.0           872000.0
                 HYC           104549.0                 NaN                NaN
                 NG            367521.0           4091000.0          3989000.0
                 PEL            71085.0            767000.0           763000.0

In [69]:
# list of NERC regions
nercs = nerc_fraction.index.get_level_values('nerc').unique()

### Group the facility data to state fuel categories and add state labels

In [70]:
cols = list(nerc_frac_match.values())
eia_fac_type = group_fuel_cats(eia_fac, state_fuel_cat)
eia_fac_type = add_facility_location(eia_fac_type, location_labels, ['state'])
eia_fac_type = eia_fac_type.groupby(['state', 'year', 'month', 'type'])[cols].sum()

In [71]:
eia_fac_type.loc[idx['OK', 2017, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
OK    2017 1     WND           982955.0           9160159.0          9160159.0
           2     WND          1053595.0           9818451.0          9818451.0
           3     WND          1237665.0          11533802.0         11533802.0
           4     WND          1164423.0          10851260.0         10851260.0
           5     WND           960103.0           8947200.0          8947200.0
           6     WND           987307.0           9200713.0          9200713.0

In [72]:
eia_fac_type.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     COW          26493.302           576574.00          520312.00
                 HYC         111786.609          1155091.03         1155091.03
                 NG          314206.726          3372210.00         3270620.00
                 PEL          66098.653           715175.00          710849.00
                 WND             86.518              893.99             893.99

### Calculate the extra gen/fuel consumption at the state levels
**Only worrying about extra generation from 2016 forward.**

In [73]:
state_extra = (state_total.loc[idx[:, 2016:, :, :], :]
               - eia_fac_type.loc[idx[:, 2016:, :, :], :])
state_extra.dropna(how='all', inplace=True)
state_extra = state_extra.reorder_levels(['year', 'state', 'month', 'type'])
state_extra.sort_index(inplace=True)
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     COW          20785.451            367385.0           306487.0
                 HYC          62929.960                 NaN                NaN
                 NG           68565.140            767269.0           767269.0
                 PEL          38760.835            400331.0           379272.0
           2     COW          19546.818            352264.0           294007.0

The huge jump in Texas state-level wind generation is why I need to use more recent lists of annual reporting facilities rather than the list from 2015.

In [74]:
state_extra.loc[idx[:, 'TX', :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 TX    1     WND           90562.51                 NaN                NaN
           2     WND          102900.59                 NaN                NaN
           3     WND          123610.89                 NaN                NaN
           4     WND          107811.34                 NaN                NaN
           5     WND          112386.52                 NaN                NaN
           6     WND           91352.99                 NaN                NaN
           7     WND          115339.50                 NaN                NaN
           8     WND           82661.79                 NaN                NaN
           9     WND           85334.20                 NaN                NaN
           10    WND          108237.19                 NaN                NaN
           11    WND           87830.32                 NaN                NaN
           12    WND           96075.55                 NaN                NaN
2017 TX    1     WND         2991690.73                 NaN                NaN
           2     WND         2924136.52                 NaN                NaN
           3     WND         3451667.09                 NaN                NaN
           4     WND         3422068.33                 NaN                NaN
           5     WND         3037395.57                 NaN                NaN
           6     WND         2438002.79                 NaN                NaN

Why is generation from facilities greater than state-level estimates in some cases?

In [75]:
state_extra.loc[(state_extra < -100).any(axis=1)]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AZ    6     HYC         -12722.670                 NaN                NaN
           7     HYC         -12463.360                 NaN                NaN
           8     HYC          -6672.920                 NaN                NaN
     CA    1     OOG         -36609.329                 NaN                NaN
           2     OOG         -35875.716                 NaN                NaN
           3     OOG         -35981.849                 NaN                NaN
           4     OOG         -25453.429                 NaN                NaN
           5     OOG         -37564.717                 NaN                NaN
           6     OOG         -36266.771                 NaN                NaN
           7     OOG         -34884.006                 NaN                NaN
           8     OOG         -35066.275                 NaN                NaN
           9     OOG         -37674.828                 NaN                NaN
           10    OOG         -36968.936                 NaN                NaN
           11    OOG         -33645.069                 NaN                NaN
           12    OOG         -36549.401                 NaN                NaN
     CT    11    PEL          -1618.145             20380.0            17560.0
     FL    1     OOG          -2215.246                 NaN                NaN
           2     OOG          -6301.215                 NaN                NaN
           3     OOG          -7743.011                 NaN                NaN
           4     OOG          -7163.809                 NaN                NaN
           5     OOG          -7527.673                 NaN                NaN
           6     OOG          -4412.662                 NaN                NaN
           7     OOG          -3398.979                 NaN                NaN
           8     OOG          -5283.892                 NaN                NaN
           9     OOG          -5375.453                 NaN                NaN
           10    OOG          -5124.908                 NaN                NaN
           11    OOG          -5939.829                 NaN                NaN
           12    OOG          -5858.501                 NaN                NaN
     IA    2     PC           -1007.471            118875.0            44503.0
           4     PC            -153.864            122387.0            61012.0
...                                 ...                 ...                ...
2017 NY    6     OOG          -9741.000                 NaN                NaN
     OH    1     WND         -54686.670                 NaN                NaN
           2     WND         -51739.260                 NaN                NaN
           3     WND         -48036.650                 NaN                NaN
           4     WND         -36701.710                 NaN                NaN
           5     OTH           -829.004                 NaN                NaN
                 WND         -43171.980                 NaN                NaN
           6     OTH           -808.997                 NaN                NaN
                 WND         -38712.580                 NaN                NaN
     OK    1     NG          -78893.944           -610495.0          -610504.0
           2     NG          -24082.120            190248.0           190242.0
     PA    1     OOG          -5416.866                 NaN                NaN
           4     OOG          -1904.495                 NaN                NaN
     RI    1     PEL          -1287.028              5968.0             5968.0
     SC    4     PEL          -5673.328             95739.0            95742.0
     UT    1     OOG           -943.999                 NaN                NaN
           2     OOG          -1026.004                 NaN                NaN
           3     OOG           -863.002                 NaN              

Sort the index of each dataframe to make sure they can be easily combined.

In [76]:
nerc_fraction.sort_index(inplace=True)
nerc_fraction.loc[idx['TX', 'WECC', :], :].tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    WECC NG        0.061815      0.056091     0.071281
           PEL       0.033746      0.008952     0.032607
           SUN       0.067911      0.067911     0.067911
           WND       0.000000      0.000000     0.000000

In [77]:
state_extra.sort_index(inplace=True)
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     COW          20785.451            367385.0           306487.0
                 HYC          62929.960                 NaN                NaN
                 NG           68565.140            767269.0           767269.0
                 PEL          38760.835            400331.0           379272.0
           2     COW          19546.818            352264.0           294007.0

Create a copy of the `nerc_fraction` dataframe with repeated values for every month of the year, so that they MultiIndex matches the `state_extra` MultiIndex

In [78]:
df_list = []
for month in range(1, 13):
    df = nerc_fraction.copy()
    df['month'] = month
    df.set_index('month', append=True, inplace=True)
    df_list.append(df)

nerc_frac_monthly = pd.concat(df_list, axis=0)
nerc_frac_monthly.sort_index(inplace=True)
nerc_frac_monthly = (nerc_frac_monthly
                     .reorder_levels(['nerc', 'state', 'month', 'type']))

In [79]:
nercs

Index(['SERC', 'SPP', 'WECC', 'MRO', 'NPCC', 'RFC', 'FRCC', 'TRE'], dtype='object', name='nerc')

Cycle through each year (2016 and 2017 in this case) and each NERC, multiplying the state-level extra generation, total fuel consumption, and fuel consumption for electricity by the share that should be allocated to each NERC.

In [80]:
df_list_outer = []
for year in [2016, 2017]:
    df_list_inner = []
    for nerc in nercs:
        df = pd.concat([(nerc_frac_monthly
                         .loc[nerc]['% generation']
                         * state_extra
                         .loc[year]['generation (mwh)']).dropna(),
                        (nerc_frac_monthly.
                         loc[nerc]['% total fuel']
                         * state_extra
                         .loc[year]['total fuel (mmbtu)']).dropna(),
                        (nerc_frac_monthly
                         .loc[nerc]['% elec fuel']
                         * state_extra
                         .loc[year]['elec fuel (mmbtu)']).dropna()],
                        axis=1)
        df.columns = nerc_frac_match.values()
        df['nerc'] = nerc
        df['year'] = year
        df = df.groupby(['year', 'nerc', 'month', 'type']).sum()
        df_list_inner.append(df)

    df_list_outer.append(pd.concat(df_list_inner))
final = pd.concat(df_list_outer)
final.sort_index(inplace=True)

In [81]:
nerc_frac_monthly.sort_index(inplace=True)

Although the dataframe is called `final`, it's really just the final allocated extra state-level generation/fuel consumption

In [82]:
final.loc[idx[2017, 'SPP', :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2017 SPP  1     WND       2.210824e+06                 NaN                NaN
          2     WND       2.235194e+06                 NaN                NaN
          3     WND       2.612558e+06                 NaN                NaN
          4     WND       2.563388e+06                 NaN                NaN
          5     WND       2.220806e+06                 NaN                NaN
          6     WND       1.839252e+06                 NaN                NaN

In [83]:
final

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2016 FRCC 1     COW       1.498682e+04        6.960830e+05       6.043759e+04
                NG        2.524146e+05        2.966904e+06       2.094634e+06
                OOG      -2.015610e+03                 NaN                NaN
                OTH       8.292912e+04                 NaN                NaN
                PEL       3.127199e+03        1.055482e+05       3.357690e+04
                SUN       2.853990e+03                 NaN                NaN
                WAS       1.303369e+05                 NaN                NaN
                WWW       6.246666e+04                 NaN                NaN
          2     COW       1.253480e+04        5.967938e+05       5.049390e+04
                NG        2.278179e+05        2.714570e+06       1.875943e+06
                OOG      -5.733355e+03                 NaN                NaN
                OTH       7.335296e+04                 NaN                NaN
                PEL       1.472412e+03        5.670722e+04       1.606330e+04
                SUN       6.826300e+03                 NaN                NaN
                WAS       1.228812e+05                 NaN                NaN
                WWW       5.591151e+04                 NaN                NaN
          3     COW       1.182164e+04        5.196626e+05       4.763910e+04
                NG        2.709514e+05        3.016059e+06       2.232633e+06
                OOG      -7.045217e+03                 NaN                NaN
                OTH       7.882161e+04                 NaN                NaN
                PEL       8.732100e+02        3.045714e+04       9.478456e+03
                SUN       8.514040e+03                 NaN                NaN
                WAS       1.269202e+05                 NaN                NaN
                WWW       5.916026e+04                 NaN                NaN
          4     COW       7.129727e+03        3.699797e+05       2.836630e+04
                NG        2.574942e+05        2.875466e+06       2.074348e+06
                OOG      -6.518213e+03                 NaN                NaN
                OTH       8.016193e+04                 NaN                NaN
                PEL       7.786320e+02        3.085405e+04       8.449949e+03
                SUN       7.350900e+03                 NaN                NaN
...                                ...                 ...                ...
2017 WECC 4     NG        1.197134e+06        1.218188e+07       1.057586e+07
                OOG      -8.100281e+04                 NaN                NaN
                OTH       2.297305e+03                 NaN                NaN
                PEL       1.269918e+03        6.154001e+04       1.513098e+04
                SUN       2.203804e+06                 NaN                NaN
                WAS       2.324025e+05                 NaN                NaN
                WND       2.946571e+06                 NaN                NaN
                WWW       3.478885e+05                 NaN                NaN
          5     COW       1.858247e+05        2.688538e+06       2.109946e+06
                GEO       7.202312e+05                 NaN                NaN
                HYC       5.549492e+06                 NaN                NaN
                NG        1.145283e+06        1.209070e+07       1.057503e+07
                OOG      -8.268553e+04                 NaN                NaN
                OTH       2.114460e+03                 NaN                NaN
                PEL       6.449952e+02        9.668846e+04       2.282808e+04
                SUN       2.547829e+06                 NaN                NaN
                WAS       2.413362e+05                 NaN                NaN
                WND       2.552278e+06                 NaN                NaN
                WWW       3.672885e+05      

In [84]:
path = join(data_path, 'NERC extra gen fuels_rev.csv')
final.to_csv(path)